Reading in verdict text sampole into python

In [91]:
import regex as re
import torch
# !pip install tiktoken
from torch.utils.data import Dataset, DataLoader


import tiktoken

In [127]:
with open('../../data/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()
    
print(f'Total number of characters: {len(raw_text)}')

print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


## create a vocabulary of words for tokenizer

In [39]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])
print(len(preprocessed))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


In [40]:
all_words = sorted(set(preprocessed))
print(len(all_words))
print(all_words[:30])

1130
['!', '"', "'", '(', ')', ',', '--', '.', ':', ';', '?', 'A', 'Ah', 'Among', 'And', 'Are', 'Arrt', 'As', 'At', 'Be', 'Begin', 'Burlington', 'But', 'By', 'Carlo', 'Chicago', 'Claude', 'Come', 'Croft', 'Destroyed']


In [41]:
vocab = {token:integer for integer , token in enumerate(all_words) }

for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 10:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)


## Create tokenizer class

In [29]:
class SimpleTokenizerV1():
    
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
        
    def encode(self , text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        
        # remove space
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        
        return ids
    
    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # replace the space if it follows after puncutaions 
        text = re.sub(r'\s+([,.?!"()\'])', r'\1' , text)
        return text

In [31]:
tokenizer = SimpleTokenizerV1(vocab)
text = """
I HAD always thought Jack Gisburn rather a cheap genius--
"""

# encoding the text 
print(tokenizer.encode(text)) 

# decoding the ids back to text
print(tokenizer.decode(tokenizer.encode(text)))


[53, 44, 149, 1003, 57, 38, 818, 115, 256, 486, 6]
I HAD always thought Jack Gisburn rather a cheap genius --


In [34]:
print(tokenizer.encode('Hello')) 

KeyError: 'Hello'

If we encounter an unknown token it errors out. To make sure our LLM doenst error out like this, we will add in special text token. 

Lets add two special token <|endoftext|> and <|unk|> lets update the vocab

In [47]:
all_words = sorted(set(preprocessed))

all_words.extend(["<|endoftext|>" , "<|unk|>"])

print(f'Length of vocab before {len(vocab.items())}')

vocab = {token:integer for integer , token in enumerate(all_words) }

print(f'Length of vocab before {len(vocab.items())}')



Length of vocab before 1130
Length of vocab before 1132


In [48]:
class SimpleTokenizerV2():
    
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
        
    def encode(self , text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        
        # remove space
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        
        # if we encounter word not in vocab add <|unk|> as its token
        
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
            
        ]
        
        ids = [self.str_to_int[s] for s in preprocessed]
        
        return ids
    
    def decode(self,ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # replace the space if it follows after puncutaions 
        text = re.sub(r'\s+([,.?!"()\'])', r'\1' , text)
        
        return text

In [55]:
tokenizer = SimpleTokenizerV2(vocab)
text1 = """
Hello , How are you ?
"""

text2 = "Do you like Tea?"

text = "<|endoftext|> ".join((text1,text2))

print(text)
# encoding the text 
print(tokenizer.encode(text)) 

# decoding the ids back to text
print(tokenizer.decode(tokenizer.encode(text)))


Hello , How are you ?
<|endoftext|> Do you like Tea?
[1131, 5, 52, 169, 1126, 10, 1130, 1131, 1126, 628, 1131, 10]
<|unk|>, How are you? <|endoftext|> <|unk|> you like <|unk|>?


#  BytePair encoding

Example
- aaabdaaabac
    - byte pair "aa" occures the most 
    - replace "aa" with "Z"
    - seq now becomes ZabdZabac
        - repeat 
        - byte pair "ab" replace with "Y"
        - seq now becomes ZYdZYac
            - now only "ac" pair is left which only occures once, so we dont need to compress it further 

Practicle example of tokenization 

{"old" : 7 , "older" : 3 , "finest" : 9 , "lowest" : 4}

Preprocessing add "<//w>"

{"old<//w>" : 7 , "older<//w>" : 3 , "finest<//w>" : 9 , "lowest<//w>" : 4}

character level tokenization 

<//w>   - 23 
o       - 14         
l       - 14
d       - 10
e       - 16
r       - 3
f       - 9
i       - 9
n       - 9
s       - 13
t       - 13
w       - 4

es appears the most 
<//w>   - 23 
o       - 14         
l       - 14
d       - 10
e       - 16 - 13 = 3
r       - 3
f       - 9
i       - 9
n       - 9
s       - 13 -13 = 0
t       - 13
w       - 4
es      - 9+4 = 13

est appears the most now 
<//w>   - 23 
o       - 14         
l       - 14
d       - 10
e       - 16 - 13 = 3
r       - 3
f       - 9
i       - 9
n       - 9
s       - 13 - 13 = 0
t       - 13 - 13 = 0
w       - 4
es      - 9+4 = 13 - 13 = 0
est     - 13

est<//w>  appears the most now 
<//w>   - 23 - 13 = 10
o       - 14         
l       - 14
d       - 10
e       - 16 - 13 = 3
r       - 3
f       - 9
i       - 9
n       - 9
s       - 13 - 13 = 0
t       - 13 - 13 = 0
w       - 4
es      - 9+4 = 13 - 13 = 0
est     - 13 - 13 = 0
est<//w>- 13




In [ ]:


tokenizer = tiktoken.get_encoding("gpt2")

In [68]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

In [69]:
print(tokenizer.encode(text , allowed_special={"<|endoftext|>"}))

print(tokenizer.decode(tokenizer.encode(text , allowed_special={"<|endoftext|>"})))

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


# Creating INPUT-TARGET pairs

In [ ]:
with open('../../data/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()
    
enc_text = tokenizer.encode(raw_text)

print(len(enc_text))

5145


In [76]:
enc_sample = enc_text[:50]

In [79]:
context_size = 4 

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f'x: {x}')
print(f'y:      {y}' )

x: [40, 367, 2885, 1464]
y:      [367, 2885, 1464, 1807]


In [82]:
for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(f'{context} ------> {desired}')

[40] ------> 367
[40, 367] ------> 2885
[40, 367, 2885] ------> 1464
[40, 367, 2885, 1464] ------> 1807


In [88]:
for i in range(1,context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "------>" ,tokenizer.decode([desired]))

I ------>  H
I H ------> AD
I HAD ------>  always
I HAD always ------>  thought


In [92]:

class GPTDatasetV1(Dataset):
    def __init__(self , txt , tokenizer , max_length , stride):
        self.input_ids = []
        self.target_ids = []
        
        # tokenize the entire text 
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        
        # using a sliding window to break the whole text into overlapping sequence of the max length
        
        for i in range(0 , len(token_ids)-max_length , stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+1+max_length]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index],self.target_ids[index]

In [93]:
def create_dataloader_v1(txt , batch_size = 4 , max_length = 256 , stride = 128 , shuffle = True , drop_last = True , num_workers = 0 ):
    
    #initialize the tokenizer 
    
    tokenizer= tiktoken.get_encoding('gpt2')
    
    #create dataset 
    dataset = GPTDatasetV1(txt , tokenizer , max_length , stride)
    
    # create dataloader 
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    
    return dataloader



Lets test the class and dataloader

In [ ]:
with open('../../data/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()
    
dataloader = create_dataloader_v1(txt = raw_text , batch_size=1 , max_length=5, stride=2 , shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)

print(first_batch)

second_batch = next(data_iter)
print(second_batch)

[tensor([[  40,  367, 2885, 1464, 1807]]), tensor([[ 367, 2885, 1464, 1807, 3619]])]
[tensor([[2885, 1464, 1807, 3619,  402]]), tensor([[1464, 1807, 3619,  402,  271]])]


In [ ]:
with open('../../data/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()
    
dataloader = create_dataloader_v1(txt = raw_text , batch_size=8 , max_length=5, stride=4 , shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)

print(first_batch)

second_batch = next(data_iter)
print(second_batch)

[tensor([[   40,   367,  2885,  1464,  1807],
        [ 1807,  3619,   402,   271, 10899],
        [10899,  2138,   257,  7026, 15632],
        [15632,   438,  2016,   257,   922],
        [  922,  5891,  1576,   438,   568],
        [  568,   340,   373,   645,  1049],
        [ 1049,  5975,   284,   502,   284],
        [  284,  3285,   326,    11,   287]]), tensor([[  367,  2885,  1464,  1807,  3619],
        [ 3619,   402,   271, 10899,  2138],
        [ 2138,   257,  7026, 15632,   438],
        [  438,  2016,   257,   922,  5891],
        [ 5891,  1576,   438,   568,   340],
        [  340,   373,   645,  1049,  5975],
        [ 5975,   284,   502,   284,  3285],
        [ 3285,   326,    11,   287,   262]])]
[tensor([[  287,   262,  6001,   286,   465],
        [  465, 13476,    11,   339,   550],
        [  550,  5710,   465, 12036,    11],
        [   11,  6405,   257,  5527, 27075],
        [27075,    11,   290,  4920,  2241],
        [ 2241,   287,   257,  4489,    64],
    

# Token Embeddings 

In [105]:
input_ids = torch.tensor([2,3,5,1])

In [106]:
vocab_size = 6 
output_dim = 3 

torch.manual_seed(123)

embedding_layer = torch.nn.Embedding(vocab_size , output_dim)

In [107]:
#random initialization weight
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [110]:
embedding_layer((input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)

# Positional Embeddings

In [ ]:
vocab_size = 50257 
output_dim = 256 

# initiate token embedding
token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

Lets intiate the data loader

In [ ]:
with open('../../data/the-verdict.txt', 'r', encoding='utf-8') as f:
    raw_text = f.read()

context_length = 4
    
dataloader = create_dataloader_v1(txt = raw_text , batch_size=8 , max_length=context_length, stride=context_length , shuffle=False)

data_iter = iter(dataloader)

inputs, targets = next(data_iter)


In [120]:
print(f'Token Id :{inputs}')
print(f'Inputs shape:{inputs.shape}')

print(f'Target Id :{targets}')
print(f'Target shape:{targets.shape}')

Token Id :tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Inputs shape:torch.Size([8, 4])
Target Id :tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
Target shape:torch.Size([8, 4])


In [121]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [124]:
# initiate position embedding
pos_embedding_layer = torch.nn.Embedding(context_length,output_dim)

pos_embeddings = pos_embedding_layer(torch.arange(context_length))

print(pos_embeddings.shape)

torch.Size([4, 256])


In [126]:
input_embedding = token_embeddings + pos_embeddings

print(input_embedding.shape)

torch.Size([8, 4, 256])
